this notebook analyzes https://github.com/leelabcnbc/thesis-yimeng-v2/blob/1874c2e8b68989e1faff68d7044f052cc8057569/scripts/feature_extraction/gaya/maskcnn_polished_with_rcnn_k_bl/submit_20200523.py

In [1]:
from os.path import dirname, relpath, realpath, join, exists

import h5py

from numpy.linalg import norm
import numpy as np

from thesis_v2 import dir_dict
from thesis_v2.submission import utils
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200523_gaya_feature_extraction_generator,
    script_keygen,
    gen_feature_extraction_global_vars
)
from json import dump, load
from os import remove

In [2]:
def generate_diff_vec(*, tensor_list, acc_mode, max_len):
    assert len(tensor_list) > 1
    
    if acc_mode=='instant':
        pass
    elif acc_mode == 'cummean':
        tensor_list_new = []
        for i in range(len(tensor_list)):
            tensor_list_new.append(np.mean(np.asarray(tensor_list[:i+1]), axis=0))
        tensor_list = tensor_list_new
    else:
        raise ValueError
    
    tensor_shape = tensor_list[0].shape
    # get norm diff
    diff_vec = np.full((max_len,), fill_value=np.nan, dtype=np.float64)
    
    for i in range(len(tensor_list)-1):
        vec_prev = tensor_list[i]
        vec_now = tensor_list[i+1]
        assert vec_prev.shape == vec_now.shape
        assert vec_prev.ndim == 3
        diff_this = norm(vec_now.ravel()-vec_prev.ravel())/norm(vec_prev.ravel())
        diff_vec[i] = diff_this
    return diff_vec

def generate_diff_vec_per_image(*, tensor_list, acc_mode, max_len):
    num_img = tensor_list[0].shape[0]
    assert num_img == 1050
    diff_vec_list = []
    for idx_img in range(num_img):
        tensor_list_this_img = [x[idx_img] for x in tensor_list]
        diff_vec_list.append(generate_diff_vec(
            tensor_list = tensor_list_this_img,
            acc_mode=acc_mode,
            max_len = max_len,
        ))
    diff_vec_list = np.asarray(diff_vec_list)
    assert diff_vec_list.shape == (num_img, max_len)
    diff_vec_mean = diff_vec_list.mean(axis=0)
    diff_vec_std = diff_vec_list.std(axis=0)
    return {
        'mean': diff_vec_mean.tolist(),
        'std': diff_vec_std.tolist(),
    }
    

def load_all():
    
    batch_key='gaya/20200523'
    global_vars_for_feature_extraction = gen_feature_extraction_global_vars(key=batch_key)
    
    for idx, param_dict in enumerate(explored_models_20200523_gaya_feature_extraction_generator()):
        
        cls = param_dict['rcnn_bl_cls']
        # only examine 5, 6, 7
        if cls <= 4:
            continue
        
        key_this = script_keygen(**param_dict)        
        h5file = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.hdf5')
        
        auxfile = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.json')
        assert exists(auxfile)
        auxfile2 = join(global_vars_for_feature_extraction['feature_file_dir'], key_this +'.per_image.json')
        assert auxfile2 != auxfile
        if not exists(auxfile2):
            with h5py.File(h5file, 'r') as f:
                g = f['test']
                resp_map = [g[f'0.{x}'][()] for x in range(cls)]
                diff_vec_map_instant_dict = generate_diff_vec_per_image(
                    tensor_list = resp_map,
                    acc_mode='instant',
                    max_len = 6,
                )

                diff_vec_map_cummean_dict = generate_diff_vec_per_image(
                    tensor_list = resp_map,
                    acc_mode='cummean',
                    max_len = 6,
                )
            # write aux file.
            try:
                with open(auxfile2, 'wt', encoding='utf-8') as f_aux:
                    dump({
                        'test': {
                            'diff_bl_stack_instant': diff_vec_map_instant_dict,
                            'diff_bl_stack_cummean': diff_vec_map_cummean_dict,
                        }
                    },
                        f_aux,
                    )
            except Exception:
                if exists(auxfile2):
                    remove(auxfile2)
                
        with open(auxfile, 'rt', encoding='utf-8') as f_aux:
            aux_obj = load(f_aux)
            
        with open(auxfile2, 'rt', encoding='utf-8') as f_aux2:
            aux_obj2 = load(f_aux2)
        
        if idx % 10 == 0:
            print(idx)
            print(aux_obj)
            print(aux_obj2)
load_all()

1160
{'test': {'diff_final_act': [0.05158759653568268, 0.03099377267062664, 0.02449006587266922, 0.02151736617088318, 0.01880582980811596, nan], 'diff_bl_stack_instant': [0.41666093468666077, 0.24323315918445587, 0.21391858160495758, 0.18863220512866974, 0.1688244342803955, nan], 'diff_bl_stack_cummean': [0.20833046734333038, 0.12163249403238297, 0.09528189152479172, 0.07707631587982178, 0.06639048457145691, nan]}}
{'test': {'diff_bl_stack_instant': {'mean': [0.43706022171747116, 0.2609968911749976, 0.23292638237987245, 0.20264216076760064, 0.1788996141013645, nan], 'std': [0.07485483839330394, 0.056111391241967117, 0.06590117999414895, 0.04805754979682413, 0.03779777980021263, nan]}, 'diff_bl_stack_cummean': {'mean': [0.21853011078777768, 0.12912480051318803, 0.10249816304161435, 0.08340234197676182, 0.07187387600541115, nan], 'std': [0.037427419863621324, 0.02737918333222497, 0.02408495777771089, 0.022144959178376868, 0.020070153856442573, nan]}}}
1170
{'test': {'diff_final_act': [0.

1460
{'test': {'diff_final_act': [0.06141998618841171, 0.036400679498910904, 0.027096576988697052, 0.0217715036123991, 0.019483987241983414, nan], 'diff_bl_stack_instant': [0.4793179929256439, 0.31753599643707275, 0.25122779607772827, 0.21809624135494232, 0.20618361234664917, nan], 'diff_bl_stack_cummean': [0.23965899646282196, 0.13722681999206543, 0.1039833351969719, 0.08372637629508972, 0.07229584455490112, nan]}}
{'test': {'diff_bl_stack_instant': {'mean': [0.48921013111159917, 0.3222276851534843, 0.25639084458351136, 0.22100021817854473, 0.21014410115423657, nan], 'std': [0.050847897903038654, 0.03068447969198514, 0.03333293821604815, 0.022783512492273818, 0.02814763844689853, nan]}, 'diff_bl_stack_cummean': {'mean': [0.24460506575448174, 0.13918932210121837, 0.10619016143537703, 0.0852775686021362, 0.07362978139803524, nan], 'std': [0.02542394917771241, 0.012742537857364454, 0.012521679888813404, 0.00947472823175399, 0.008232120508252339, nan]}}}
1470
{'test': {'diff_final_act': [

1810
{'test': {'diff_final_act': [0.06804998964071274, 0.03406907990574837, 0.024052586406469345, 0.01822737604379654, 0.015353373251855373, 0.013516725972294807], 'diff_bl_stack_instant': [0.5445720553398132, 0.37530577182769775, 0.3130178451538086, 0.2847442924976349, 0.2706567347049713, 0.2671985924243927], 'diff_bl_stack_cummean': [0.2722860276699066, 0.1322421133518219, 0.09013216942548752, 0.06850304454565048, 0.058240219950675964, 0.05187322199344635]}}
{'test': {'diff_bl_stack_instant': {'mean': [0.531591660834494, 0.3675783292736326, 0.3101371534097762, 0.2853588249853679, 0.2732587794249966, 0.2709624316294988], 'std': [0.06836031924537933, 0.0690064673496818, 0.06264040592089246, 0.06048089190572465, 0.06023772313900411, 0.05758793940537582]}, 'diff_bl_stack_cummean': {'mean': [0.2657958303888639, 0.13063266387297995, 0.08988742989798387, 0.06842276777185145, 0.05825249489219416, 0.05189245561758677], 'std': [0.03418015961352017, 0.014748878675335416, 0.007917540178804822, 0